# Proyecto Final Área Imágenes

In [ ]:
import numpy as np
import os 
import io
from skimage import local_binary_pattern


In [ ]:
from keras.applications.vgg16 import VGG16
from keras.layers import *
from keras.models import Model
#from keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from skimage import transform
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Embedding
from keras import backend as K


def convnet_model_():
    vgg_model = VGG16(weights=None, include_top=False)
    x = vgg_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.6)(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.6)(x)
    x = Lambda(lambda  x: K.l2_normalize(x,axis=1))(x)
    convnet_model = Model(inputs=vgg_model.input, outputs=x)
    return convnet_model

def convnet_model():
	vgg_model = VGG16(weights=None, include_top=False)
	x = vgg_model.output
	x - GlobalAveragePooling2D()(x)
	x = Dense(4096, activation='relu')(x)
	x = Dropout(0.6)(x)
	x = Dense(4096, activation='relu')(x)
	x = Dropout(0.6)(x)

def deep_rank_model():

    convnet_model = convnet_model_()
    first_input = Input(shape=(224,224,3))
    first_conv = Conv2D(96, kernel_size=(8, 8),strides=(16,16), padding='same')(first_input)
    first_max = MaxPool2D(pool_size=(3,3),strides = (4,4),padding='same')(first_conv)
    first_max = Flatten()(first_max)
    first_max = Lambda(lambda  x: K.l2_normalize(x,axis=1))(first_max)

    second_input = Input(shape=(224,224,3))
    second_conv = Conv2D(96, kernel_size=(8, 8),strides=(32,32), padding='same')(second_input)
    second_max = MaxPool2D(pool_size=(7,7),strides = (2,2),padding='same')(second_conv)
    second_max = Flatten()(second_max)
    second_max = Lambda(lambda  x: K.l2_normalize(x,axis=1))(second_max)

    merge_one = concatenate([first_max, second_max])

    merge_two = concatenate([merge_one, convnet_model.output])
    emb = Dense(4096)(merge_two)
    l2_norm_final = Lambda(lambda  x: K.l2_normalize(x,axis=1))(emb)

    final_model = Model(inputs=[first_input, second_input, convnet_model.input], outputs=l2_norm_final)

    return final_model


model = deep_rank_model()

for layer in model.layers:
    print (layer.name, layer.output_shape)


if not os.path.exists("./deepranking-v2-150000.h5"):
    print("No se encontró el modelo, se descargará")
    import gdown
    url = 'https://drive.google.com/uc?id=1TmUKqp_TnzSP0TeAHIyTv8jG4KZeNqQP'
    output = 'deepranking-v2-150000.h5'
    gdown.download(url, output, quiet=False)

model.load_weights("./deepranking-v2-150000.h5")

In [ ]:
IMG_DATASET = []

In [ ]:
def load(path):
    """Loads images from path"""
    imgs = []
    names = os.listdir(path)

    for name in names: 
        image = io.imread(fname=os.path.join(path,name))
        imgs.append(image) 
    return np.array(imgs)

### Parte 1
Vectores característicos

In [ ]:
def classic_method(image):
    "Implements LBP algorithm "
    lbp = local_binary_pattern(image, P=8, R=1)
    #segmentar
    lbp = lbp[:500,:600] 
    M = (lbp.shape[0])//5
    N = (lbp.shape[1])//5
    tiles = [lbp[x:x+M,y:y+N] for x in range(0,lbp.shape[0],M) for y in range(0,lbp.shape[1],N)]
        
    #segment histogramas  
    tiles_histogram = []
    #calcular el histograma para cada segmento
    for i in tiles: 
        hist = np.histogram(i.ravel(),density=True,bins = 59,range=(0, 59))[0]
        tiles_histogram.append(hist)
    full_histogram = np.array(tiles_histogram).flatten()
    return np.array(full_histogram) 
    


def CNN_method(image1):
    image1 = img_to_array(image1).astype("float64")
    image1 = transform.resize(image1, (224, 224))
    image1 *= 1. / 255
    image1 = np.expand_dims(image1, axis = 0)
    return model.predict([image1, image1, image1])[0]

def get_vector(image, extractor_type):
    
    if extractor_type == 'classic':
        vector = classic_method(image)
    else:
        vector =CNN_method(image)
    
    return vector

### Parte 2

In [ ]:
def similarity_metric(vec_1,vec_2, measure = 'euclidean'):
    if measure == 'euclidean':
        resta = vec_1 - vec_2
        return np.sqrt(np.sum(resta**2))
    elif measure == 'manhattan':
        resta = vec_1 - vec_2
        return np.sum(np.abs(resta))
    elif measure == 'cosine':
        dot_product = np.dot(vec_1, vec_2)
        norm_a = np.linalg.norm(vec_1)
        norm_b = np.linalg.norm(vec_2)
        cosine_similarity = dot_product / (norm_a * norm_b)
        return 1 - cosine_similarity
    else:
        raise ValueError(f"Unknown measure: {measure}")


### Parte 3

In [ ]:
def compare(image, img_database):
    pass




### Parte 4

In [ ]:
def relevance_ranking(compare_list):
    n = len(compare_list)
    rank = (1/n)* np.sum(compare_list)


### Parte 5

In [ ]:
def normalized_rank(RETURN_FUNCION_ARRIBA,method):
    pass